In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os

import os
os.chdir('/home/cat/code/manifolds/')

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange

from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


#
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


#
from tsnecuda import TSNE

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds


In [5]:
####################################################################
#################### LOAD DLC BODY COORDINATES #####################
####################################################################

import csv

fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/DON-005099_20210609_TR-BSL_S1-ACQ_DLC.csv'

def load_csv(fname):

    from numpy import genfromtxt
    locs = genfromtxt(fname, 
                      delimiter=',', 
                      dtype='str')

    print (locs.shape)
    print (locs)
    
    return locs


locs = load_csv(fname)
body_feature_idx = 5*3+1
print ("Body feature index", np.arange(body_feature_idx,body_feature_idx+2,1))
neck = np.float32(locs[3:,body_feature_idx:body_feature_idx+2])
print (neck.shape)

np.save('/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/locs.npy', neck)

(36003, 43)
[['scorer' 'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000'
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000' ...
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000'
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000'
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000']
 ['bodyparts' 'snout' 'snout' ... 'tail_tip' 'tail_tip' 'tail_tip']
 ['coords' 'x' 'y' ... 'x' 'y' 'likelihood']
 ...
 ['35997' '1929.4710693359375' '1526.892822265625' ... '1547.64892578125'
  '1165.356689453125' '0.9995512366294861']
 ['35998' '1929.422607421875' '1524.6185302734375' ...
  '1547.0325927734375' '1169.79541015625' '0.999924898147583']
 ['35999' '1929.53369140625' '1525.818603515625' ... '1547.0889892578125'
  '1175.59375' '0.9989499449729919']]
Body feature index [16 17]
(36000, 2)


In [6]:
print (neck)

[[1935.7167      -1.1136727]
 [1935.6301      -0.8825569]
 [1936.2708      -0.5595889]
 ...
 [1483.8932    1211.9296   ]
 [1485.0629    1211.3875   ]
 [1486.7317    1211.2448   ]]


In [7]:
####################################################################
################### LOAD FLUORESCENCE TRACES #######################
####################################################################
fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/DON-005099_20210609_TRACES_ACQ.csv'

def load_traces(fname):
    
    from tqdm import tqdm, trange
    
    from numpy import genfromtxt
    data = genfromtxt(fname, delimiter=',', dtype='str')

    cells = np.float32(data[2:,1:]).T
    times = np.float32(data[2:,0])
    
    
    print ("F cells: ", cells.shape, "times: ", times.shape)
    return cells, times

F, F_times = load_traces(fname)


F cells:  (203, 36026) times:  (36026,)


In [9]:
####################################################################
################## LOAD DECONVOLVES SPIKES #########################
####################################################################

def find_nearest_times(times1, times2):

    # 
    idx3 = []
    for k in range(times2.shape[0]):
        spk = times2[k]
        time = np.argmin(np.abs(times1-spk))
        idx3.append(time)
        
    return idx3
    
#
def load_spks(fname, F, F_times):
    
    from tqdm import tqdm, trange
    
    from numpy import genfromtxt
    data = genfromtxt(fname, delimiter=',', dtype='str')
    print (data)
    
    # 
    times = np.float32(data[1:,0])
    cells = data[1:,1]
    probs = np.float32(data[1:,2])
    
    #
    cell_ids = np.sort(np.unique(cells))
    
    
    spks = np.zeros((F.shape[0], F.shape[1]),'float32')
    ctr=0
    for cell_id in tqdm(cell_ids):

        # find cell indexes
        idx = np.where(cells==cell_id)[0]
    
        # grab probabilities at indexes
        temp_probs = probs[idx]
    
        # get cell times
        temp_time = times[idx]
        
        # convert cell times to 
        times_idx = find_nearest_times(F_times, temp_time)
        
        # insert probabilities
        spks[ctr,times_idx] = temp_probs
        
        #
        ctr+=1       
    
    return spks

#
fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/DON-005099_20210609_DECSPK_ACQ.csv'
spks = load_spks(fname, F, F_times)
print (spks.shape)



[['Time (s)' ' Cell Name' ' Value']
 ['0.049961' ' C000' ' 0.713386']
 ['1.149103' ' C000' ' 0.4412095']
 ...
 ['1799.295454' ' C202' ' 3.850622']
 ['1799.445337' ' C202' ' 0.6576414']
 ['1799.59522' ' C202' ' 4.768372e-07']]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [01:17<00:00,  2.62it/s]

(203, 36026)


In [10]:
# np.save('/media/cat/4TB/donato/laurenz/F.npy',F)
# np.save('/media/cat/4TB/donato/laurenz/spks.npy',F)

iscell = np.ones((F.shape[0],2))
np.save('/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/iscell.npy',iscell)

In [11]:
#######################################################
#######################################################
#######################################################
plt.plot(spks[0], label='oasis spikes')
plt.plot(F[0], label='raw fluorescence')
plt.legend()
plt.show()